In [ ]:
from gym import envs
all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print(sorted(env_ids))

In [ ]:
import numpy as np
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete), \
            "Should only be used to wrap Discrete envs."
        self.n = self.observation_space.n
        self.observation_space = gym.spaces.Box(0, 1, (self.n,))
    
    def observation(self, obs):
        new_obs = np.zeros(self.n)
        new_obs[obs] = 1
        return new_obs

In [ ]:


import gym
env = ObservationWrapper(gym.make('Taxi-v3'))
#env = gym.make('Taxi-v3')
print(env.action_space)
print(env.observation_space)



env = gym.make('CartPole-v0')
print(env.action_space)
print(env.observation_space)





In [8]:
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
env=jss_lite(instance_path='resources/jsp_instances/standard/abz5.txt')

for _ in range(50):
    token=False
    state=env.reset()
    for i in range(150):
        legal_action=env.get_legal_actions(state)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        if action > env.action_space.n/2:
            token=True
            #print("before:")
            #env.render(start_count=1,x_bar="Job",y_bar="Machine")
            state, reward, done, info=env.step(action)


        else:
            state, reward, done, info=env.step(action)
            if token:
                token=False
                #print("after:")
                #env.render(start_count=1,x_bar="Job",y_bar="Machine")
        #print(action)
        #tmp=(env.processed_and_max_time_job_matrix)
        #print(env.count_finished_tasks_machine_matrix)
        #print(env.production_list)
        #env.render()
        if done==True:
            print(f"finished! no legal actions possible after {i+1} steps")
            break
    #env.render(start_count=1,x_bar="Job",y_bar="Machine")



finished! no legal actions possible after 130 steps


finished! no legal actions possible after 132 steps


finished! no legal actions possible after 123 steps


finished! no legal actions possible after 137 steps


finished! no legal actions possible after 143 steps


...

finished! no legal actions possible after 123 steps


finished! no legal actions possible after 133 steps


finished! no legal actions possible after 124 steps


finished! no legal actions possible after 129 steps


KeyboardInterrupt: 

In [6]:
import plotly.express as px
x_bar="Machine"
y_bar="Job"
df.sort_values(inplace=True,by="Job")
fig = px.timeline(df, x_start="Start", x_end="Finish", color=x_bar, y=y_bar)

fig.show()

In [4]:
print(action)

print(env.get_legal_actions(state))
print(env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]])
cur_machine=env.current_machines_status
job_ma=env.job_machine_matrix
print(env.count_finished_tasks_job_matrix[action])
#print(env.current_machines_status[env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]],:])
#print(env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]])
#print(env.current_machines_status)
env.render()


10
[False False False False False False False False False False False]


IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
import ale_py as ale
ale.importroms('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/roms/Space Invaders.bin')
import gym
env = gym.make('SpaceInvaders-v4')
env.reset()
env.render()


In [3]:
import JSSEnv
import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
import src.jsp_instance_parser 
import time
import gym
import pandas as pd
import random
import numpy as np
from src.jss_lite.jss_lite import jss_lite

index=['klagenfurt','alex','lite']
df=pd.DataFrame(np.zeros((3,5)),columns=['setuptime','resettime','50xactiontime','col4','col5'],index=index)
#df=pd.DataFrame
#df.set_index('index')
for i in range(100):

    # def timer(innerfun):
    #     tmp_t=time.time()
    #     tmp_r=innerfun()
    #     return time.time()-tmp_t,tmp_r

    # tmp_t=time.time()
    # env2=gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/abz8.txt'})
    # t_1=(time.time()-tmp_t)
    # df['setuptime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    env_3=jss_lite(instance_path='resources/jsp_instances/standard/abz8.txt')
    t_1=time.time()-tmp_t
    df['setuptime']['lite']+=t_1/100




    # tmp_t=time.time()
    # path='resources/jsp_instances/standard/abz8.txt'
    # curr_instance=src.jsp_instance_parser.parse_jps_standard_specification(path)
    # res,std_matrix=curr_instance
    # env_alex = jss_env.DisjunctiveGraphJssEnv(res,default_visualisations='gantt_console')
    # t_2=(time.time()-tmp_t)
    # df['setuptime']['alex']+=t_2/100
    # tmp_t=time.time()
    # t=env2.reset()
    # t_1=(time.time()-tmp_t)
    # df['resettime']['klagenfurt']+=t_1/100
    # tmp_t=time.time()
    # t2=env_alex.reset()
    # t_2=(time.time()-tmp_t)
    # df['resettime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env_3.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['lite']+=t_1/100


    # state=env2.reset()
    # for i in range(50):
    #     legal_action=state['action_mask']
    #     action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
    #     tmp_t=time.time()
    #     state, reward, done, info=env2.step(action)
    #     t_1=(time.time()-tmp_t)
    #     df['50xactiontime']['klagenfurt']+=t_1/50

    #     if done==True:
    #         print(f"finished! no legal actions possible after {i} steps")
    #         break
    state=env_3.reset()
    for i in range(50):
        legal_action=env_3.get_legal_actions(state)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env_3.step(action)
        t_1=(time.time()-tmp_t)
        df['50xactiontime']['lite']+=t_1/50

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break



    # action_list=np.arange(0,env_alex.n_jobs*env_alex.n_machines)
    # for i in range(50):
    #     action_mask=env_alex.valid_action_mask()
    #     actions=action_list[action_mask]
    #     next_action=random.choice(actions)
    #     #next_action=actions[action_mask[0]]
    #     #print(next_action)
    #     tmp_t=time.time()
    #     state,reward,done,info =env_alex.step(next_action)
    #     t_2=(time.time()-tmp_t)
    #     df['50xactiontime']['alex']+=t_2/50
    #     if done == True:
    #         print(f"finished after {i} steps")
    #         break
        #env.render()


print(df)



            setuptime  resettime  50xactiontime  col4  col5
klagenfurt   0.000000   0.000000       0.000000   0.0   0.0
alex         0.000000   0.000000       0.000000   0.0   0.0
lite         0.001373   0.000762       0.107003   0.0   0.0


In [ ]:
import resource
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
dt = datetime(2022, 1, 1, 0, 0, 0)
#stores quatuple(job,task,start_time,finish_time) 
time_now=time.time()
production_list=env_3.production_list
def production_to_dict(input,i,j):
    return(dict(Task=f"job_{input[0]}", Start=dt + timedelta(seconds=int(input[2])),Finish=dt + timedelta(seconds=int(input[3])),Resource=f"machine_{i}"))
df=pd.DataFrame()
liste=[]
for i in range(10):
    for j in range(10):
        if production_list[i,j] is not None:
            liste.append(production_to_dict(production_list[i,j],i,j))

df_2=pd.DataFrame(liste)
fig = px.timeline(df_2, x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.show()

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])

#fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
#fig.show()